In [ ]:
# This needs to be rewritten. Currently clips are taken and analyzed in 2 second chunks. 
# Audio needs to come in as a stream. Every X number of milliseconds OR after the FFT/Prediction is completed, the next
# 2048 samples need to be taken from the stream and analyzed. 

# https://forum.pjrc.com/threads/24492-Using-the-PDB-on-Teensy-3#post_message_37659
# https://forum.pjrc.com/threads/24611-Problem-with-DMA-interrupt
# https://forum.pjrc.com/threads/18237-teensy-3-0-memory-to-memory-DMA-help

# https://python-sounddevice.readthedocs.io/en/0.3.11/examples.html

In [1]:
import tinn
import random
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import Queue
from threading import Timer
from threading import Thread
from threading import Event

In [2]:
#https://pymotw.com/2/threading/

In [3]:
class Data:
    def __init__(self, path, nips, nops):
        self.read_data(path, nips, nops)

    def __repr__(self):
        return str( str(self) +'rows with'+ str(len(self.in_[0])) +'inputs and'+ str(len(self.tg[0])) +'outputs' )
        #return f'{len(self)} rows with {len(self.in_[0])} inputs and {len(self.tg[0])} outputs.'

    def read_data(self, path, nips, nops):
        self.in_, self.tg = [], []
        with open(path) as data_file:
            for line in data_file:
                row = list(map(float, line.split()))
                self.in_.append(row[:nips])
                self.tg.append(row[nips:])

    def shuffle(self):
        indexes = list(range(len(self.in_)))
        random.shuffle(indexes)
        self.in_ = [self.in_[i] for i in indexes]
        self.tg = [self.tg[i] for i in indexes]

    def __len__(self):
        return len(self.in_)
    
    def dLen(self):
        return len(self.in_)

In [4]:
class DataOnTheFly:
    def __init__(self, arr, nips, nops):
        self.read_array(arr, nips, nops)
        
    def read_array(self, arr, nips, nops):
        self.in_, self.tg = [], []

        self.in_.append(arr[:nips])
        self.tg.append(arr[nips:])
        

In [5]:
def train(_nips, _nhid, _nops, _rate, _anneal, _data, _iter):
    nips = _nips
    nhid = _nhid
    nops = _nops
    rate = _rate
    anneal = _anneal
    
    data = Data(_data, nips, nops)
    
    t = tinn.Tinn(nips, nhid, nops)
    
    for i in range(_iter):
        data.shuffle()
        error = 0
        for in_, tg in zip(data.in_, data.tg):
            error += tinn.xttrain(t, in_, tg, rate)
        print 'Error: ' + str(error/len(data)) + ' learning rate: ' + str(rate) + ' Iteration: ' + str(i)
        rate *= anneal

    t.save('savedApp.tinn')

In [6]:
def predict(arr, _nips, _nops):

    data = DataOnTheFly(arr, _nips, _nops)
    
    loaded = tinn.xtload('savedApp.tinn')

    in_ = data.in_[0]
    tg = data.tg[0]
        
    pd = tinn.xtpredict(loaded, in_)
        
    #print(' '.join(map(str, tg)))
    #print(' '.join(map(str, pd)))
    
    return pd

In [7]:
#https://stackoverflow.com/questions/12435211/python-threading-timer-repeat-function-every-n-seconds
#Repeated Timer
class MyThread(Thread):
    def __init__(self, event):
        Thread.__init__(self)
        self.stopped = event

    def run(self):
        #while not self.stopped.wait(0.1):
        while True:
            #Call a function
            fastThread()

In [8]:
def getAudio():
    fs = 1024
    duration = 2
    
    myRecording1 = sd.rec(duration * fs, samplerate=fs, channels=1)
    sd.wait()
    
    #Each sample is of type list. Convert from list to int
    bClip = []
    for i in range(len(myRecording1)):
        bClip.append( myRecording1[i][0] )
    
    return bClip

In [9]:
def fastThread():
    #Get clip
    aClip = getAudio()
    
    #Push to queue
    processQueue.put( aClip )
    
    #Maybe delay a split second - Done in Thread Class

In [10]:
def getFFT(_aClip):
    
    fs = 1024
    duration = 2
    
    #Get data
    #data = aClips[i]
    data = _aClip

    #Smooth the FFT by windowing data
    data = data * np.hanning(len(data))

    #Preform FFT and take only real data
    fft = abs(np.fft.fft(data).real)

    #Keep only first half
    fft = fft[:int(len(fft))/2]

    #Create Frequency list
    freq = np.fft.fftfreq( (fs * duration) ,1.0/fs)

    #Keep only first half
    freq = freq[:int(len(freq))/2] 

    #At this point, the length of the fft/freq is cut in reduced from 2048 to 1024. But the cut off frequency is actually 512.
    #The freq axis shows why. Values are now 0, 0.5, 1, 1.5Hz...
    #Therefore drop every other value
    fft = fft[0::2]
    freq = freq[0::2]

    #Save for later
    #ffts.append( fft )
    
    return fft

In [11]:
def formatDataForPrediction(arr):
    #Convert to list
    arr = arr.tolist()
    
    #Append 1 0 to list - woof
    arr = arr + [1.0, 0.0]
    
    return arr

In [12]:
#Train NN
train(512, 14, 2, 0.5, 0.99, 'woofsAndMeows.data', 32)

Error: 0.279490370668 learning rate: 0.5 Iteration: 0
Error: 0.195405499718 learning rate: 0.495 Iteration: 1
Error: 0.140789940615 learning rate: 0.49005 Iteration: 2
Error: 0.0987608685429 learning rate: 0.4851495 Iteration: 3
Error: 0.0679380945633 learning rate: 0.480298005 Iteration: 4
Error: 0.047077918646 learning rate: 0.47549502495 Iteration: 5
Error: 0.0340747725038 learning rate: 0.4707400747 Iteration: 6
Error: 0.025777947391 learning rate: 0.466032673953 Iteration: 7
Error: 0.020437512278 learning rate: 0.461372347214 Iteration: 8
Error: 0.0167813825426 learning rate: 0.456758623742 Iteration: 9
Error: 0.0140296872466 learning rate: 0.452191037504 Iteration: 10
Error: 0.0119693885029 learning rate: 0.447669127129 Iteration: 11
Error: 0.010462614755 learning rate: 0.443192435858 Iteration: 12
Error: 0.00924990163876 learning rate: 0.438760511499 Iteration: 13
Error: 0.00825115374045 learning rate: 0.434372906384 Iteration: 14
Error: 0.00744721151007 learning rate: 0.4300291

In [13]:
#Create queues
processQueue = Queue.Queue()

#Create events
stopFlag = Event()

#Create threads
thread = MyThread(stopFlag)
thread.start()

# this will stop the timer
#stopFlag.set()

In [ ]:
#While loop
while True:
    
    #Wait for something in Queue
    if not processQueue.empty(): 
        aClip = processQueue.get()

        #Get FFT
        clipFFT = getFFT( aClip )
        
        #Format for NN
        formattedClip = formatDataForPrediction(clipFFT)

        ##Predict
        predictedValues = predict( formattedClip, 512, 2 )
        print predictedValues[0], predictedValues[1]

        #Found woof
        if predictedValues[0] > 0.8 and predictedValues[1] < 0.04:
            print 'Found Woof'
        #Found meow   
        elif predictedValues[0] > 0.4 and predictedValues[1] > 0.7:
            print 'Meow'


0.838885410336 0.159673219903
0.804584290029 0.193554066376
0.81924213066 0.179717793191
0.824047011727 0.174338027576
0.719581161584 0.279767348191
0.732706795322 0.266171672955
0.783775341867 0.215170317437
0.721207805377 0.27749587083
0.777292797544 0.221368471081
0.654308370382 0.345564405152
0.742741817957 0.256398760574
0.862886993071 0.136171459601
0.804609301485 0.194077794745
0.86416485787 0.135003245547
0.861792520056 0.137108695199
0.701000429289 0.298949899467
0.858228515929 0.140386239822
0.71123337797 0.287532258293
0.73956415889 0.25994321325
0.640716182026 0.358536735215
0.775667314741 0.222972469776
0.793435565036 0.205273560038
0.762656830328 0.236085151828
0.784201575384 0.214962390573
0.771289400397 0.227367594595
0.693576841366 0.296610211317
0.658493309906 0.34009496714
0.864162200019 0.134851455389
0.804214667458 0.194654506393
0.746963094513 0.251026690279
0.775920118628 0.222903186706
0.658461036692 0.339636615267
0.747078963922 0.251200974592
0.837951991018 0.

0.713152788126 0.28640231519
0.857931792617 0.141524128896
0.819076913017 0.180267479386
0.860608231532 0.138674776429
0.720199364913 0.27908973935
0.856226544613 0.142821136388
0.704771232161 0.294473228839
0.77424929358 0.225310940965
0.848147141504 0.150795263142
0.797082522207 0.201649599281
0.770096482511 0.229580394001
0.805922427688 0.193812244368
0.788273999567 0.211817043581
0.792213543556 0.207040509392
0.837310462245 0.162179355749
0.654721403924 0.344716908859
0.751105845779 0.247911868066
0.778055543753 0.221210585262
0.730110804187 0.269415968988
0.785233430321 0.214864452789
0.828361670475 0.171340254644
0.745862236725 0.253833292574
0.786636093249 0.212772495374
0.765273497419 0.234614120629
0.741154014422 0.258538058028
0.710861678572 0.289222639037
0.74080764114 0.258406648308
0.805936538216 0.193502854602
0.793700297849 0.205702484924
0.719388910448 0.280458406691
0.786764045681 0.211607284131
0.646000351883 0.352718466431
0.731517964884 0.268320549439
0.706759465938

0.472104969972 0.524143722285
0.542604733826 0.457521827118
0.771840398406 0.227429565579
0.299738079946 0.698843302188
0.628883202847 0.369443846906
0.472426670587 0.526356534114
0.689473850751 0.30908235812
0.804662842381 0.192430853494
0.606989665778 0.391364616793
0.675560704404 0.323341204345
0.501995663322 0.496494859623
0.718871086579 0.279446041601
0.616753588508 0.382143550602
0.521581317941 0.472133835927
0.592142095559 0.404174051529
0.55629194081 0.443039345645
0.781993027978 0.216250992295
0.593971682099 0.401424566346
0.349905674896 0.648848648456
0.608349269518 0.390031678864
0.684215729351 0.313710302293
0.686949301895 0.311369403421
0.780834626789 0.218118832001
0.654253876421 0.34520454847
0.585789532582 0.412412353739
0.676722093327 0.321712196589
0.708837078302 0.290358177624
0.656533237188 0.342466221859
0.866699087765 0.131686434099
0.613359483708 0.385620149455
0.769937550173 0.229755358631
0.622449740687 0.376497511814
0.716248357945 0.282708692332
0.65361083709

0.742854720343 0.258208935003
0.729003205729 0.271684734047
0.747171395512 0.253673264453
0.65527580397 0.345762657323
0.809801799568 0.190351071973
0.716407099913 0.284048066521
0.803418329263 0.196612500493
0.618455153002 0.381020122395
0.562484653928 0.438636467315
0.668139630566 0.333249474125
0.6846790205 0.316040051663
0.747055305684 0.252938426857
0.723786130673 0.276060349024
0.68242968859 0.317593541283
0.588334491774 0.412096523933
0.602368868051 0.39927235672
0.751571042434 0.24843240544
0.584205504526 0.417367339606
0.766126543255 0.233861879084
0.582950443045 0.417845346804
0.653690850166 0.346296539078
0.694068313908 0.306738419699
0.766888740309 0.23310811236
0.627573551425 0.371841996648
0.69566610999 0.304342781505
0.671265970539 0.329396733495
0.755260143156 0.2446396555
0.806955438081 0.193201864836
0.634363185596 0.366881139109
0.713908831808 0.286942088682
0.687481227749 0.313415103681
0.687345245288 0.313766416636
0.696824524523 0.303275076488
0.786597938308 0.213

0.646780058315 0.353786328626
0.698142384861 0.302546518518
0.693379240889 0.306679365648
0.675468228846 0.325104051005
0.760729620155 0.239396445732
0.698998838226 0.301768844015
0.69047527845 0.310049145427
0.714555374445 0.285757926536
0.750894244218 0.249703230663
0.754933445248 0.24533549915
0.709476707371 0.291389311833
0.7394449546 0.261086078115
0.570035169527 0.431714805207
0.747779886804 0.25294621311
0.80011951421 0.200168293717
0.684302192545 0.316805989485
0.624632255198 0.376421011424
0.73424640493 0.266649287725
0.658191944397 0.342997286529
0.646369744069 0.35538088667
0.603433936396 0.39857161706
0.513953705162 0.48774349898
0.729389596568 0.271140731996
0.76071608245 0.240043896467
0.774679547287 0.225971191784
0.757556522338 0.243097843215
0.704895317974 0.296182323912
0.652461283857 0.348098377565
0.716996796695 0.283812460333
0.796315304596 0.20385668166
0.675647782988 0.325637851062
0.641791915831 0.359135194693
0.711669961074 0.28918280432
0.780900790278 0.219598

...

0.815216061379 0.185445280582
0.793088762979 0.207744454935
0.780821633403 0.220355214652
0.768354250805 0.232689409549
0.816859624251 0.183609164528
0.787481417876 0.213516456949
0.841160527453 0.15916578224
0.746523245132 0.254867362291
0.856593732324 0.14387242962
0.815701232671 0.184981899682
0.801660838892 0.199239380076
0.796849694015 0.203814390533
0.874366492682 0.125644915691
0.788606176891 0.212321744078
0.825638564089 0.174606064969
0.84817736098 0.15225568952
0.783419321577 0.217998057682
0.848250509083 0.152128002802
0.730528612575 0.270919115947
0.899832235766 0.100227313405
0.826680521679 0.173908818616
0.845873425818 0.154542357687
0.834689613605 0.166078964964
0.811775976472 0.188830597053
0.823808752074 0.17691592309
0.845010433694 0.155420556705
0.840614568298 0.159529099931
0.868055447623 0.13199145228
0.769865774377 0.230788667105
0.816034846039 0.182953664571
0.729599149156 0.271380324015
0.810384972455 0.190514809037
0.782048070876 0.218653410131
0.853602362022 0

0.809174886704 0.191477802191
0.798957093281 0.202033416778
0.804396943205 0.1964507777
0.824010491242 0.176496377686
0.84343910721 0.15676359508
0.630929659707 0.371233250287
0.774515272185 0.226383277919
0.774489613294 0.225355137281
0.830531115255 0.169944421385
0.792390843562 0.208267172174
0.814922077491 0.185853352052
0.831695703568 0.16892881609
0.77519949477 0.226096339128
0.829151977251 0.171237396163
0.850152139089 0.150127470102
0.828899961276 0.171780252364
0.798658659176 0.20192386653
0.752180126409 0.249077874148
0.794838637225 0.205948508697
0.770169098312 0.230883387955
0.719842293784 0.281496983301
0.802578028024 0.197832727201
0.784644262241 0.216218848644
0.762069665226 0.239039892356
0.811636678101 0.188792894027
0.793487621908 0.207240268689
0.807803332362 0.192492377563
0.863136317237 0.137052203231
0.793169872715 0.207645223884
0.788939729743 0.211911643584
0.838966574328 0.161756782926
0.787578187542 0.213475967148
0.812089588782 0.18614760463
0.86230228763 0.13

0.79825058294 0.202526099792
0.834869154336 0.165776315667
0.796057912893 0.20498966661
0.803120915818 0.197675043324
0.851085236358 0.14921080147
0.82587596779 0.174888343972
0.739592526211 0.261578247798
0.832156891025 0.168138685244
0.793218217254 0.207144655301
0.784088269186 0.216946132616
0.825101466486 0.175533761786
0.839096753846 0.161377427717
0.735236086701 0.265247420281
0.765657445982 0.235215150787
0.852479495145 0.147728507683
0.831164617848 0.169419877528
0.792229589337 0.208328349905
0.857835326754 0.142324590143
0.877425108995 0.122629048979
0.839482544532 0.160867159528
0.785231738962 0.215494321395
0.805740408615 0.194826592225
0.841983711297 0.157999487716
0.827712391585 0.172946485031
0.84430058113 0.156151652771
0.843648992557 0.156904853084
0.790688109091 0.210189318615
0.808190260904 0.192940833499
0.828884330708 0.171645156863
0.894126054682 0.105986834891
0.79758351761 0.203369198216
0.847582536172 0.152851155791
0.831791965197 0.168600309248
0.82639299004 0.

0.830900674086 0.169761814888
0.802772198107 0.198226024077
0.812906936376 0.187898855202
0.885677311867 0.114551314273
0.798502715634 0.20221881767
0.828351348462 0.172430263123
0.837357284729 0.163089635958
0.891231360522 0.10890119162
0.75161143966 0.24961119738
0.767921004971 0.233338881241
0.804080505078 0.19700125671
0.804633205557 0.196015025988
0.818446991231 0.182233606428
0.846005377231 0.154334688305
0.818513365627 0.182308041069
0.800508575637 0.200532891073
0.83453204831 0.165857521395
0.858463125112 0.141826042423
0.804003856673 0.196548738195
0.756372847084 0.244966630617
0.837165710931 0.163241873815
0.858823210719 0.141485604177
0.86430067281 0.134823644843
0.889118559525 0.111092435492
0.776284332397 0.224378725133
0.757423189226 0.244139643056
0.82805353177 0.172628963599
0.815215250035 0.185407029584
0.840359921574 0.160206931502
0.776786032643 0.224188969012
0.829318043749 0.170978100266
0.771778257868 0.229280239901
0.755642608441 0.245559364881
0.802880060056 0.1

0.758409969149 0.243436623894
0.809259499109 0.191924373852
0.800794149922 0.200741530764
0.814843185242 0.18632701757
0.857066780979 0.143640932158
0.736685093174 0.265315734222
0.797334153465 0.203830999599
0.760046186481 0.241259881892
0.788485686044 0.212835114954
0.762789925674 0.239126227742
0.605729481101 0.394964959851
0.813239676928 0.187791077596
0.778999608713 0.222416598994
0.869958184932 0.13055570084
0.78382354265 0.21784557182
0.783803054356 0.217722698639
0.814253457067 0.187063677025
0.72300163749 0.279108257608
0.624627725035 0.378516057941
0.802632932205 0.198894210777
0.831828838168 0.169252860196
0.792038798029 0.209477321117
0.813215154789 0.187997608463
0.851110738588 0.149356729939
0.830345106445 0.170978324754
0.78246634152 0.21939673841
0.740185567191 0.262319134476
0.731564253554 0.270965319252
0.699332776709 0.302773830585
0.751880586785 0.249704773926
0.743172033082 0.259001163375
0.751732645993 0.250270405325
0.771163142202 0.230540280552
0.737570609993 0.